In [1]:
cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [2]:
cd Photo_Mosaic/

/content/drive/MyDrive/Photo_Mosaic


In [3]:
import os, random
from PIL import Image
import numpy as np


In [41]:
base_dir = '/content/drive/MyDrive/Photo_Mosaic'
images_dir = base_dir + "/data/Trees"
target_dir = "/content/drive/MyDrive/Linh xinh/IMG_0477 (1).JPG"
grid_size = (100,100)


In [42]:
def get_images(images_dir):
  images = []
  image_paths = os.listdir(images_dir)
  for path in image_paths:
    try:
      path = os.path.abspath(os.path.join(images_dir, path))
      fb = open(path, "rb")
      img = Image.open(fb)
      images.append(img)
      img.load()
      fb.close()
    except:
      print("Invalid Image")
  return images



In [43]:
target_image = Image.open(target_dir)
db_images = get_images(images_dir)
random.shuffle(db_images)
reuse_image = True
resize_image = True

if not reuse_image:
  if(grid_size[0]*grid_size[1] >= len(db_images)):
    print("Not enough data in database")
    exit()
if resize_image:
  dims = (int(target_image.size[0]/grid_size[0]), int(target_image.size[1]/grid_size[1]))
  for img in db_images:
    img.thumbnail(dims)

In [44]:
def split_image(img, grid_size):
  W,H= img.size[0], img.size[1]
  m,n  = grid_size
  w = int(W/m)
  h = int(H/n)
  imgs = []
  for i in range(0, m):
    for j in range(0, n):
      imgs.append(img.crop((i*w, j*h, (i+1)*w, (j+1)*h)))
  return imgs
def get_averageRBG(img):
  img = np.array(img)
  W, H, D = img.shape
  return tuple(np.average(img.reshape(W*H, D), axis=0))
def get_match_index(target_avg, avgs):
  min_dist = float("inf")
  index = 0
  for i,avg in enumerate(avgs):
    dist = ((avg[0]-target_avg[0])*(avg[0]-target_avg[0]) + 
    (avg[1]-target_avg[1])*(avg[1]-target_avg[1])+
    (avg[2]-target_avg[2])*(avg[2]-target_avg[2]))
    if dist < min_dist:
      min_dist = dist
      index = i
  return index
def create_from_grids(out_images, grid_size):
  m,n = grid_size
  width = max([img.size[0] for img in out_images])
  height = max([img.size[1] for img in out_images])
  grid_img = Image.new("RGB", (n*width, m*height))
  for i, img in enumerate(out_images):
    row = int(i/n)
    col = i - n*row
    grid_img.paste(img, (col*width, row*height ))
  return grid_img
def create_mosaic(target_image, db_images, grid_size, reuse_image=True):
  target_images = split_image(target_image, grid_size)
  avgs = []
  out_images = []
  for img in db_images:
    try:
      avgs.append(get_averageRBG(img))
    except ValueError:
      continue
  for i, img in enumerate(target_images):
    avg = get_averageRBG(img)
    index = get_match_index(avg, avgs)
    out_images.append(db_images[index])
    if i%10==0:
      print("Processing {}/{} images".format(i, len(target_images)))
    if not reuse_image:
      db_images.remove(index)
  mosaic = create_from_grids(out_images, grid_size)
  return mosaic
  
  


In [45]:
mosaic_image = create_mosaic(target_image, db_images, grid_size)
mosaic_image.save("./out.jpg", 'jpeg')

Processing 0/10000 images
Processing 10/10000 images
Processing 20/10000 images
Processing 30/10000 images
Processing 40/10000 images
Processing 50/10000 images
Processing 60/10000 images
Processing 70/10000 images
Processing 80/10000 images
Processing 90/10000 images
Processing 100/10000 images
Processing 110/10000 images
Processing 120/10000 images
Processing 130/10000 images
Processing 140/10000 images
Processing 150/10000 images
Processing 160/10000 images
Processing 170/10000 images
Processing 180/10000 images
Processing 190/10000 images
Processing 200/10000 images
Processing 210/10000 images
Processing 220/10000 images
Processing 230/10000 images
Processing 240/10000 images
Processing 250/10000 images
Processing 260/10000 images
Processing 270/10000 images
Processing 280/10000 images
Processing 290/10000 images
Processing 300/10000 images
Processing 310/10000 images
Processing 320/10000 images
Processing 330/10000 images
Processing 340/10000 images
Processing 350/10000 images
Pro

In [46]:
target_image.show()

In [18]:
target_image.size

(3024, 4032)